In [1]:
%run 'ks01_Functions_only.ipynb'
notebook_prefix = 'ks16'
image_counter = Counter()

/Users/karen/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Users/karen/anaconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


### Relative >>> absolute position

In [ ]:
position_translation = pd.read_table(files_dump_folder + 'position_translation.csv')
position_translation.set_index('relative_position', inplace=True)

def get_absolute_position(segment_number, relative_position):
    return int(position_translation.iloc[relative_position]['S'+str(segment_number)])

### Reading data on sign epistasis from Lucas

In [22]:
lucas_sign_epistasis = pd.read_csv(files_dump_folder + 'sign_epistasis/' + 'lucas_sign_epistasis.csv')
lucas_reciprocal_sign_epistasis = pd.read_csv(files_dump_folder + 'sign_epistasis/' + 'lucas_reciprocal_sign_epistasis.csv')

p_value_threshold = 0.01
filtered = lucas_sign_epistasis[lucas_sign_epistasis['pBon'] < p_value_threshold]
sign_epistasis_positions = set.union(set(filtered['VarPos_absolute'].values), 
                                                set(filtered['SubPos_absolute'].values))
sign_epistasis_positions = sorted([int(s) for s in sign_epistasis_positions])
print len(sign_epistasis_positions), 'positions under sign epistasis'

reciprocal_sign_epistasis_positions = set.union(set(lucas_reciprocal_sign_epistasis['position1'].values), 
                                                set(lucas_reciprocal_sign_epistasis['position1'].values))
reciprocal_sign_epistasis_positions = sorted([int(s) for s in reciprocal_sign_epistasis_positions])
print len(reciprocal_sign_epistasis_positions), 'positions under reciprocal sign epistasis'

94 positions under sign epistasis
59 positions under reciprocal sign epistasis


### Reading 24-mer structure

In [29]:
his3_aligned_to_4lom_assembly_file = os.path.join(structure_predictions_folder, 'his3_24mer_assembly', 
                                                  'his3_swiss_aligned_to_4lom_assembly.pdb')
structure = Bio.PDB.PDBParser().get_structure('his3_swiss_assembly', his3_aligned_to_4lom_assembly_file)
model = structure[0]

/Users/karen/anaconda/lib/python2.7/site-packages/Bio/PDB/StructureBuilder.py:85: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 37657.
  PDBConstructionWarning)
/Users/karen/anaconda/lib/python2.7/site-packages/Bio/PDB/StructureBuilder.py:85: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 37675.
  PDBConstructionWarning)
/Users/karen/anaconda/lib/python2.7/site-packages/Bio/PDB/StructureBuilder.py:85: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 37693.
  PDBConstructionWarning)
/Users/karen/anaconda/lib/python2.7/site-packages/Bio/PDB/StructureBuilder.py:85: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 37711.
  PDBConstructionWarning)
/Users/karen/anaconda/lib/python2.7/site-packages/Bio/PDB/StructureBuilder.py:85: PDBConstructionWarning: WARNING: Chain E is discontinuous at line 37729.
  PDBConstructionWarning)
/Users/karen/anaconda/lib/python2.7/site-packages/Bio/PDB/StructureBuilder.py:85: PDBConstructi

In [ ]:
substrates = [r for chain in model.get_chains() for r in chain if r.get_resname() == 'IYP']
mn = [r for chain in model.get_chains() for r in chain if r.get_resname() == ' MN']

In [30]:
constant_chain = model['A']
positions_in_crystal = [r.id[1] for r in constant_chain.get_residues() if r.get_resname() in aa3]

for residue in constant_chain.get_residues():
    if residue.get_resname() in aa3:
        for position in positions_in_crystal:
            get_d
        for other_chain in model.get_chains():

<Residue THR het=  resseq=2 icode= >
<Residue GLU het=  resseq=3 icode= >
<Residue GLN het=  resseq=4 icode= >
<Residue LYS het=  resseq=5 icode= >
<Residue ALA het=  resseq=6 icode= >
<Residue LEU het=  resseq=7 icode= >
<Residue VAL het=  resseq=8 icode= >
<Residue LYS het=  resseq=9 icode= >
<Residue ARG het=  resseq=10 icode= >
<Residue ILE het=  resseq=11 icode= >
<Residue THR het=  resseq=12 icode= >
<Residue ASN het=  resseq=13 icode= >
<Residue GLU het=  resseq=14 icode= >
<Residue THR het=  resseq=15 icode= >
<Residue LYS het=  resseq=16 icode= >
<Residue ILE het=  resseq=17 icode= >
<Residue GLN het=  resseq=18 icode= >
<Residue ILE het=  resseq=19 icode= >
<Residue ALA het=  resseq=20 icode= >
<Residue ILE het=  resseq=21 icode= >
<Residue SER het=  resseq=22 icode= >
<Residue LEU het=  resseq=23 icode= >
<Residue LYS het=  resseq=24 icode= >
<Residue GLY het=  resseq=25 icode= >
<Residue GLY het=  resseq=26 icode= >
<Residue PRO het=  resseq=27 icode= >
<Residue LEU het=  r